1) Data Collection and Exploratory Data Analysis (EDA)
Data collection and EDA can be easily performed with Python due to the libraries used for data analytics, such as Numpy and Pandas. Hence, you will start by importing these libraries along with other visualization and statistical libraries that helped you during the later steps.

In [ ]:
# importing pandas for reading data and performing other dataframe-related operations
import pandas as pd

# importing numpy for performing various numerical operations
import numpy as np

# importing matplotlib, seaborn and plotly for visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
from plotly.subplots
import make_subplots

# importing statistical libraries
import math
import scipy.stats as stats
import statsmodels.api as sm

Once done, you imported the data as a Pandas DataFrame and created a copy for data cleaning and downstream analytical steps.


In [ ]:

# importing the sales data as a pandas DataFrame
sales_data = pd.read_csv('sales_info.csv')

# creating a copy of the data for analysis
df = sales_data.copy()



The first obvious step was to examine the data, so you performed exploratory data analysis using Python. You dealt with sales data containing information about a company’s various products across multiple categories and countries. In addition to the product and transaction details, you also provided customer demographic information.


In [ ]:

# viewing the first few rows of the data
df.head()

After reviewing the data, you created a table describing each column and how it could be used during data analysis.

Next, you calculated the data dimensions and found that you dealt with around 3.5K transactions and 19 columns.

In [ ]:
# finding the number of rows and columns there in the data
print("The number of rows are {} and the number of columns are {}".format(df.shape[0],df.shape[1]))

As you had a basic understanding of the data, it was time to start with data cleaning. Data cleaning needs to focus on the following key aspects-

Basic understanding of data
Duplicates
Incorrect Data Types
Anomalies/Outliers
Missing Values
Structural Errors
As you have already understood the data, start by checking for duplicates.



2) Data Cleaning
Data cleaning is a crucial process in data analytics. It improves data quality, ensuring the insights gained from the analytical process are accurate, consistent, complete, reliable, and reproducible.

Data can have errors, biases, and incomplete information, so numerous data-cleaning steps are performed before data analysis. This section will show you the various data-cleaning steps.

a. Duplicate Removal

It found that there were eight duplicate rows in the data.

b. Type Casting

Next, you check the data types in the columns. Several columns, such as `Date` and `Customer Estimated Income,` seemed to have the wrong data types.
You will use drop.duplicated() command to remove them and ensure that the duplicates had gone.

In [ ]:

#A. checking if there are any duplicates
df.duplicated().sum()

# removing duplicate rows
df = df.drop_duplicates()

# re-checking if there are any duplicates
df.duplicated().sum()


#B. finding the data types and number of non-null values in the data
df.info()

To understand why that is the case, you view and explore the unique values of all the columns. While for `Customer Estimated Income`, the presence of the $ symbol caused wrong data types, several other issues were present, such as invalid values masking missing values and inconsistent data (e.g., `Education`).

In [ ]:
# need to find what all unwanted values are present in the different columns that are causing it to have incorrect datatypes

# first creating a copy of the data that does not have missing values
df_without_na = df.dropna().copy()

# finding the unique values for each column in the data to find such unwanted values
for i in df_without_na:
print('\n', i, df_without_na[i].unique())

To resolve the invalid values:
1. note them down and replace them with missing ones so they can be dealt with through imputation.
2. You remove the $ symbol from Income and change its data type to float.
3. You also changed the data types of the remaining columns to their correct types, as the invalid values no longer caused any hindrance. Upon re-checking, all data types seemed to be correct.


In [ ]:
#1. listing invalid values to be replaced with np.nan
invalid_values = ["%48224*#(", "(#23#(@!", "!!@#$%^&*", "??><<{}[]", "ABCDE", "NANANANA", "-", "???"]

# replacing invalid values with np.nan
df = df.replace(invalid_values, np.nan)

#2. removing the '$' sign from values in Estimated Income column and converting it to numeric.
df['Customer Estimated Income'] = df['Customer Estimated Income'].replace('[\$,]', '', regex=True).astype(float)

#3. typecasting columns to their correct data types
df['Loyalty Balance'] = pd.to_numeric(df['Loyalty Balance'])
df['Date'] = pd.to_datetime(df['Date'])

# rechecking column data types
df.dtypes

c. Anomaly/Outlier Treatment

1. To detect outliers, you create a function that creates boxplots for specified columns.
2. You will then create boxplots for the numeric columns. Upon inspection, you found that while a few columns had extreme/abnormally large values (as expected in sales data), some columns had anomalous values (e.g., `Customer Estimated Income` and `Customer Duration` with negative values).

In [ ]:
#1. creating function for boxplots
def plot_boxplots(df, columns, num_cols=3):

# calculating number of rows and columns for subplots
num_rows = math.ceil(len(columns) / num_cols)

# creating subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, num_rows * 6))

# flattening the axes array for easy iteration
axes = axes.flatten()

# ploting boxplots for each specified column
for i, col in enumerate(columns):
sns.boxplot(data=df, y=col, ax=axes[i])
axes[i].set_title(f'Boxplot of {col}')

# removing empty subplots
for j in range(i + 1, len(axes)):
fig.delaxes(axes[j])

# showing plot
plt.tight_layout()
plt.show()

#2. selecting numerical columns
numerical_cols = df.drop('Year', axis=1).select_dtypes(include='number').columns

# creating boxplots
plot_boxplots(df, numerical_cols)

To handle such anomalies, you create a function that caps outliers using the Inter-Quartile (IQR) method.

In [ ]:
# creating function to perform outlier capping
def iqr_outlier_capping(df, columns):

# creating a copy of the DataFrame to avoid modifying the original data
df_capped = df.copy()
for col in columns

    # calculating Q1 (25th percentile) and Q3 (75th percentile)
    Q1 = df_capped[col].quantile(0.25)
    Q3 = df_capped[col].quantile(0.75)

    # calculating IQR (Interquartile Range)
    IQR = Q3 - Q1

    # calculating lower and upper bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # capping the outliers
    df_capped[col] = df_capped[col].clip(lower=lower_bound, upper=upper_bound)

    return df_capped

Next, you will use the above-created function on columns with anomalies and recreate the boxplots for them to ensure that the anomalies were indeed gone.

In [ ]:
# performing outlier capping with anomalous values
columns_to_cap = ['Customer Duration', 'Loyalty Balance', 'Customer Estimated Income']
df = iqr_outlier_capping(df, columns_to_cap)

# re-creating boxplots for the columns where outlier capping is performed
plot_boxplots(df, columns_to_cap)

d. Missing Value Imputation

One of the most crucial aspects of data cleaning is dealing with missing values. As you have already dealt with invalid values that were masking missing values, you now have a much better idea of the status of missing values in the data. Upon detection, you found several columns with missing values.
You create a function that performs group-level missing value imputations using mean and mode for imputing numerical and categorical columns, respectively. 

In [ ]:
# printing column names with missing value
print(df.isna().sum()[df.isna().sum()>0].index)

# creating a function for imputing missing values in a target column using group-level mean/mode values
def impute_missing_values(df, target_column, group_by_column, column_type):

# making a copy of the dataframe to avoid modifying the original dataframe
df_imputed = df.copy()

# ensuring target_column and group_by_column exist in the dataframe
if target_column not in df_imputed.columns or group_by_column not in df_imputed.columns:
    raise ValueError("Specified columns are not present in the dataframe.")

# imputing based on column type
if column_type == 'numeric':


    # checking if the target column is numeric
    if not pd.api.types.is_numeric_dtype(df_imputed[target_column]):

        raise TypeError(f"Column '{target_column}' is not numeric. Please specify 'categorical' for non-numeric columns.")
    

    # computing the mean value for each group and fill missing values
    group_means = df_imputed.groupby(group_by_column)[target_column].transform('mean')
    df_imputed[target_column] = df_imputed[target_column].fillna(group_means)
elif column_type == 'categorical':

    # checking if the target column is categorical
    if pd.api.types.is_numeric_dtype(df_imputed[target_column]):
        raise TypeError(f"Column '{target_column}' is numeric. Please specify 'numeric' for numeric columns.")

    # computing the mode value for each group and fill missing values
    mode_values = df_imputed.groupby(group_by_column)[target_column].transform(lambda x: x.mode()[0] if not x.mode().empty else np.nan)
    df_imputed[target_column] = df_imputed[target_column].fillna(mode_values)
else:
    raise ValueError("Invalid column type specified. Use 'numeric' or 'categorical'.")
return df_imputed



In [ ]:
You then used the function to impute missing values and used appropriate group columns. Upon re-checking, it became evident that the data was free of missing values.

# using the function above to perform missing value imputation for numerical columns
df = impute_missing_values(df, target_column = 'Customer Estimated Income', group_by_column = 'State', column_type = 'numeric')
df = impute_missing_values(df, target_column = 'Loyalty Balance', group_by_column = 'Country', column_type = 'numeric')
df = impute_missing_values(df, target_column = 'Customer Duration', group_by_column = 'Customer Gender', column_type = 'numeric')

# using the function above to perform missing value imputation for categorical columns
df = impute_missing_values(df, target_column = 'Customer Education Level', group_by_column = 'Customer Gender', column_type = 'categorical')
df = impute_missing_values(df, target_column = 'Customer Marital Status', group_by_column = 'Country', column_type = 'categorical')

# rechecking if there are any missing values left
df.isna().sum()


e. Fixing Inconsistent Data

During typecasting, you found some structural issues in the data with the `Customer Education Level` column having inconsistent categories. Upon carefully looking at the categories and their counts, you found that some categories are mentioned differently (e.g., PhD being also mentioned as Research Degree and Doctorate).
To resolve this, you removed the inconsistent categories by standardizing them, eventually providing you with only four education levels.

# checking the different categories in the Education Level column
df['Customer Education Level'].value_counts().sort_index()

# mapping for standardizing categorical values
education_mapping = {
'Bachelor':'Bachelor',
'Doctorate':'PhD',
'Graduate Degree':'Master',
'High School':'High School',
'Master':'Master',
'Masters Diploma':'Master',
'PhD':'PhD',
'Research Degree':'PhD',
'Senior Secondary Education':'High School',
'Undergrad':'Bachelor',
'Undergraduate Degree':'Bachelor'
}

# standardizing the 'Customer Education Level' column
df['Customer Education Level'] = df['Customer Education Level'].map(education_mapping)

# rechecking if the categories have been fixed
df['Customer Education Level'].unique()

As the data is now clean, it’s time to analyze it. To perform data analysis using Python, you must have your data as a Pandas DataFrame, as it is much easier to deal with. As you already had the data in this form, you had multiple ways to analyze data.

In Python, you have data manipulation libraries like Pandas and Numpy, visualization libraries like Matplotlib, Seaborn, and Plotly, and statistical libraries like Scipy and Statsmodels. All such libraries allow you to look at the data from various perspectives and uncover hidden patterns.

The following sections will show all the libraries, functions, and techniques for slicing and diceing the data. Note that the focus will not be on the insights gained from the analysis, as discussed in the ‘How to communicate your insights’ section, which involves reporting. Therefore, the aim will be to understand how Python can be leveraged and the approach to be taken when performing data analysis.

4) Data Mining
a. The simplest and easiest way to analyze data is to mine it. This involves extracting key valuable information about the columns. Data analytics with Python is easy because of libraries like Pandas. If you have your data as a Pandas DataFrame, you can easily use the describe function, which provides statistical information about the columns.
b. The same function can also be applied to finding key insights about the categorical columns.
c. It was necessary to derive a few important variables to mine the data further. These included calculating the year-month combination of the transaction, profit, profit margin, etc.


In [ ]:
#A. finding the key statistical values of the numerical columns
df.describe().T

#B. # finding the key statistical values of the categorical columns
df.drop('Date', axis=1).describe(exclude=np.number).T 

#C. Creating a year-month variable for easily understanding the date related columns
# dropping the year and month column as this information is already there in the Date column
df = df.drop(['Year','Month'],axis=1)

# extracting year and creating a Year column
df['Year'] = df['Date'].dt.year

# extracting year and month and creating a Year-Month column
df['Year_Month'] = df['Date'].dt.strftime('%Y-%m')

# calculating profit
df['Profit'] = df['Revenue'] - df['Cost']

# calculating unit profit
df['Unit Profit']=df['Unit Price']-df['Unit Cost']

# calculating profit margin
df['Profit Margins'] = df['Profit'] / df['Revenue']

# calculating unit profit margin
df['Unit Profit Margin']= df['Unit Profit']/df['Unit Price']



Once such key variables become available, key performance metrics (KPI) can be easily calculated using Python, as performing arithmetic operations between different columns is extremely easy with Pandas.

In [ ]:
# Saes KPIs
ttl_revenue = df['Revenue'].sum()
ttl_profit = df['Profit'].sum()
ttl_revenue_previous = df[df['Year'] == 2015]['Revenue'].sum()
ttl_revenue_current = df[df['Year'] == 2016]['Revenue'].sum()
rga = (ttl_revenue_current-ttl_revenue_previous)/ttl_revenue_previous
pm = (ttl_profit/ttl_revenue)*100
aov = ttl_revenue/len(df)

# Customer KPIs
avg_age = df['Customer Age'].mean()
median_income = df['Customer Estimated Income'].median()
mode_martl_status = df['Customer Marital Status'].mode()[0]
mode_edu_level = df['Customer Education Level'].mode()[0]
male_percentage = (df[df['Customer Gender'] == 'M'].shape[0] / df.shape[0]) * 100
female_percentage = (df[df['Customer Gender'] == 'F'].shape[0] / df.shape[0]) * 100

# Product KPIs
top_selling_prod = df.groupby('Sub Category')['Revenue'].sum().sort_values(ascending=False).index[0]
most_purchased_prod = df.groupby('Sub Category')['Quantity'].sum().sort_values(ascending=False).index[0]
most_costly_prod = df.groupby('Sub Category')['Cost'].sum().sort_values(ascending=False).index[0]
most_profitable_prod = df.groupby('Sub Category')['Profit'].sum().sort_values(ascending=False).index[0]
most_margin_prod = df.groupby('Sub Category')['Profit Margins'].sum().sort_values(ascending=False).index[0]

# Operational KPIs
ttl_cost = df['Cost'].sum()
avg_basket_size = df['Quantity'].mean()
avg_profit_margin = df['Profit Margins'].mean()
top_costly_country = df.groupby('Country')['Cost'].sum().sort_values(ascending=False).index[0]

# Print statements for KPIs
print(f"Total Revenue: ${ttl_revenue:,.2f}")
print(f"Total Profit: ${ttl_profit:,.2f}")
print(f"Revenue Growth Rate (2015 to 2016): {rga:.2%}")
print(f"Profit Margin: {pm:.2f}%")
print(f"Average Order Value: ${aov:,.2f}")

print("\nCustomer KPIs:")
print(f"Average Customer Age: {avg_age:.1f} years")
print(f"Median Customer Estimated Income: ${median_income:,.2f}")
print(f"Most Common Marital Status: {mode_martl_status}")
print(f"Most Common Education Level: {mode_edu_level}")
print(f"Percentage of Male Customers: {male_percentage:.2f}%")
print(f"Percentage of Female Customers: {female_percentage:.2f}%")

print("\nProduct KPIs:")
print(f"Top Selling Product Sub-Category: {top_selling_prod}")
print(f"Most Purchased Product Sub-Category: {most_purchased_prod}")
print(f"Most Costly Product Sub-Category: {most_costly_prod}")
print(f"Most Profitable Product Sub-Category: {most_profitable_prod}")
print(f"Product Sub-Category with Highest Profit Margin: {most_margin_prod}")

print("\nOperational KPIs:")
print(f"Total Cost: ${ttl_cost:,.2f}")
print(f"Average Basket Size: {avg_basket_size:.2f} items")
print(f"Average Profit Margin: {avg_profit_margin:.2f}%")
print(f"Country with Highest Total Cost: {top_costly_country}")

You’ll now do some complex analysis and calculate crucial financial information for different products. You will do this by grouping the dataframe by product categories and sub-categories and aggregating it by calculating the sum of the key financial columns like revenue, cost, profit, and margin.


In [ ]:
# grouping data by product category and sub-category and aggregating by revenue and cost by summing them
prod_analysis_df = df.groupby(['Product Category','Sub Category'])[['Revenue','Cost']].sum().reset_index()

# calculating profit and profit margin
prod_analysis_df['Profits']= prod_analysis_df['Revenue'] - prod_analysis_df['Cost']
prod_analysis_df['Profit Margins']= prod_analysis_df['Profits']/ prod_analysis_df['Revenue']

# viewing data
prod_analysis_df

Similarly, you can also calculate this information and more for different countries.

In [ ]:
# grouping data by country with specific aggregation functions for each column
aggregation_functions = {
'Revenue': 'sum',
'Cost': 'sum',
'Quantity': 'sum',
'Profit': 'sum',
'Profit Margins': 'mean'
}
country_analysis_df = df.groupby('Country').agg(aggregation_functions).reset_index()

# calculating the total sums for the columns to get proportions
total_sums = country_analysis_df[['Revenue', 'Cost', 'Quantity', 'Profit', 'Profit Margins']].sum()

# calculating proportions and adding them as new columns
country_analysis_df['Revenue Proportion'] = round(country_analysis_df['Revenue'] / total_sums['Revenue'],3)*100
country_analysis_df['Cost Proportion'] = round(country_analysis_df['Cost'] / total_sums['Cost'],3)*100
country_analysis_df['Quantity Proportion'] = round(country_analysis_df['Quantity'] / total_sums['Quantity'],3)*100
country_analysis_df['Profit Proportion'] = round(country_analysis_df['Profit'] / total_sums['Profit'],3)*100
country_analysis_df['Profit Margins Proportion'] = round(country_analysis_df['Profit Margins'] / total_sums['Profit Margins'],3)*100

# rearranging columns to place similar columns next to each other
country_analysis_df = country_analysis_df[['Country',
'Revenue', 'Revenue Proportion',
'Cost', 'Cost Proportion',
'Quantity', 'Quantity Proportion',
'Profit', 'Profit Proportion',
'Profit Margins', 'Profit Margins Proportion']]

# viewing the data
country_analysis_df